In [ ]:
import sys
import xlsxwriter # pip install XlsxWriter
import requests # pip install requests
from bs4 import BeautifulSoup as bs # pip install beautifulsoup4

headers = {
            'accept': '*/*', 
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
            }
vacancy = input('Укажите название вакансии: ')
base_url = f'https://hh.ru/search/vacancy?area=1&search_period=30&text={vacancy}&page=' 
                                # area=1 - Москва, search_period=30 - За 30 последних дней, Data science
pages = int(input('Укажите кол-во страниц для парсинга: '))
jobs =[]

def hh_parse(base_url, headers):
        zero = 0
        while pages > zero:
                zero = str(zero)
                session = requests.Session()
                request = session.get(base_url + zero, headers = headers)
                if request.status_code == 200:
                        soup = bs(request.content, 'html.parser')
                        divs = soup.find_all('div', attrs = {'data-qa': 'vacancy-serp__vacancy'})
                        for div in divs:
                                title = div.find('a', attrs = {'data-qa': 'vacancy-serp__vacancy-title'}).text
                                compensation = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'})
                                if compensation == None: # Если зарплата не указана
                                        compensation = 'None'
                                else:
                                        compensation = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'}).text
                                href = div.find('a', attrs = {'data-qa': 'vacancy-serp__vacancy-title'})['href']
                                try:
                                        company = div.find('a', attrs = {'data-qa': 'vacancy-serp__vacancy-employer'}).text
                                except:
                                        company = 'None'
                                text1 = div.find('div', attrs = {'data-qa': 'vacancy-serp__vacancy_snippet_responsibility'}).text
                                text2 = div.find('div', attrs = {'data-qa': 'vacancy-serp__vacancy_snippet_requirement'}).text
                                content = text1 + '  ' + text2
                                all_txt = [title, compensation, company, content, href]
                                jobs.append(all_txt)
                        zero = int(zero)
                        zero += 1

                else:
                        print('error')

                # Запись в Excel файл
                workbook = xlsxwriter.Workbook('Vacancy.xlsx')
                worksheet = workbook.add_worksheet()
                # Добавим стили форматирования
                bold = workbook.add_format({'bold': 1})
                bold.set_align('center')
                center_H_V = workbook.add_format()
                center_H_V.set_align('center')
                center_H_V.set_align('vcenter')
                center_V = workbook.add_format()
                center_V.set_align('vcenter')
                cell_wrap = workbook.add_format()
                cell_wrap.set_text_wrap()

                # Настройка ширины колонок
                worksheet.set_column(0, 0, 35)  # A  https://xlsxwriter.readthedocs.io/worksheet.html#set_column
                worksheet.set_column(1, 1, 20) # B
                worksheet.set_column(2, 2, 40) # C
                worksheet.set_column(3, 3, 135) # D
                worksheet.set_column(4, 4, 45) # E

                worksheet.write('A1', 'Наименование', bold)
                worksheet.write('B1', 'Зарплата', bold)
                worksheet.write('C1', 'Компания', bold)
                worksheet.write('D1', 'Описание', bold)
                worksheet.write('E1', 'Ссылка', bold)

                row = 1
                col = 0
                for i in jobs:
                        worksheet.write_string (row, col, i[0], center_V)
                        worksheet.write_string (row, col + 1, i[1], center_H_V)
                        worksheet.write_string (row, col + 2, i[2], center_H_V)
                        worksheet.write_string (row, col + 3, i[3], cell_wrap)
                        # worksheet.write_url (row, col + 4, i[4], center_H_V) 
                        worksheet.write_url (row, col + 4, i[4])
                        row += 1

                print('OK')
        workbook.close()

hh_parse(base_url, headers)